In [8]:

pip install redial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 634 kB 7.9 MB/s 
  Created wheel for urwid: filename=urwid-2.1.2-cp37-cp37m-linux_x86_64.whl size=258328 sha256=56c24de60e54c406f477b34707a58db2ee7a7a6c9cddc580c1cdb37586db53bc
  Stored in directory: /root/.cache/pip/wheels/79/77/cf/cae9cf1cc3f1f777f9db531424bbd9e15aa38e4ca28dbe499e
Successfully built urwid


In [ ]:
# !pip install sentence_transformers

In [3]:

import nltk
# nltk.download('punkt')
nltk.download('all')
# nltk.download('wordnet')
# nltk.download('stopwords')
from copy import deepcopy
import itertools
import json
import pandas
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
import pandas as pd
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from numpy.linalg import norm

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [4]:


base_src='./drive/MyDrive'
def init(path, debug):
    train_data = []
    for line in open(f"{path}/train_data.json", "r"):
        train_data.append(json.loads(line))

    if debug:
        print(f"Loaded {len(train_data)} train conversations")

    test_data = []
    for line in open(f"{path}/test_data.json", "r"):
        test_data.append(json.loads(line))

    if debug:
        print(f"Loaded {len(test_data)} test conversations")

    mention_dataframe = pandas.read_csv(f"{path}/movies_with_mentions.csv")

    if debug:
        print(f"Loaded {len(mention_dataframe.values)} movies with mentions")

    return train_data, test_data, mention_dataframe
    
class RedialParser:
    def __init__(self, base_src='./drive/MyDrive', debug=False):
        self.train, self.test, self.movie = init(base_src, debug)

        self.__train = deepcopy(self.train)
        self.__test = deepcopy(self.test)
        self.__movie = deepcopy(self.movie)

        self.__model = None

    def Restore(self):
        """
        TODO: Restore train, test, and movie dataset to initial state
        """
        self.train = deepcopy(self.__train)
        self.test = deepcopy(self.__test)
        self.movie = deepcopy(self.__movie)

    def Movies(self, train=True) -> dict:
        """
        TODO: Get movie list in dataset
            :arg
                train (bool): Target dataset, (train=True, test=False, all=None)
            :return
                dict: {index, MovieName}
        """
        if train is None:
            result = self.Movies()
            result.update(self.Movies(False))
            return result

        target = None
        if train is True:
            target = self.train
        elif train is False:
            target = self.test

        result = {}

        if target is not None:
            for elem in target:
                result.update(elem['movieMentions'])

        return result

    def describe(self):
        """
        TODO: Describe its datasets
        """
        len1, len2 = len(self.train), len(self.test)
        n1, n2 = 0, 0
        m1, m2 = 0, 0

        for e in self.train:
            n1 += len(e['movieMentions'])
            m1 += len(e['messages'])
        for e in self.test:
            n2 += len(e['movieMentions'])
            m2 += len(e['messages'])

        print('Brief information:\n'
              f'Length of train data: {len1}\n'
              f'Length of test data: {len2}\n\n'
              'Data information:\n'
              f'Key parameters: {list(self.train[0].keys())}\n'
              f'Key parameters in Questions: {list(list(self.train[0]["respondentQuestions"].values())[0].keys())}\n'
              f'Key parameters in messages: {list(self.train[0]["messages"][0].keys())}\n\n'
              'Context information:\n'
              f'Total mentioned movie number (train): {n1}\n'
              f'Total mentioned movie number in unique (train): {len(self.Movies())}\n'
              f'Total message number (train): {m1}\n'
              f'Total mentioned movie number (test): {n2}\n'
              f'Total mentioned movie number in unique (test): {len(self.Movies(False))}\n'
              f'Total message number (test): {m2}\n'
              f'Average mentioned movie numbers per conversation (train): {n1 / len1}\n'
              f'Average message numbers per conversation (train): {m1 / len1}\n'
              f'Average mentioned movie numbers per conversation (test): {n2 / len2}\n'
              f'Average message numbers per conversation (test): {m2 / len2}\n\n'
              , end='')


In [11]:
# 특수문자 제거하기
compile = re.compile("\W+")
line3=[]
name=[]
if __name__ == "__main__":
    parser = RedialParser()
    parser.describe()

    num=len(parser.train)
    print(num)
    movienum=[[] for _ in range(10)]
    line2 = [[] for _ in range(10)]
    print(line2)
    for i in range(10):
        print(f'Train {i}')
        for msg in parser.train[i]['messages']:
            print(f'{msg["senderWorkerId"]}: {msg["text"]}')
            # print(msg["text"])
            line2[i].append(msg["text"])
            line3.append(msg["text"])
        for j in range(len(line2[i])):
          a = compile.sub(" ",line2[i][j])
          line2[i][j] = a.lower()
          numbers = re.findall(r'\d+', line2[i][j])
          for k in range(len(numbers)):
            if len(numbers[k])>=4:
              movienum[i].append(numbers[k])
              name.append(numbers[k])
            

        print()

print(line2)
print(movienum)



Brief information:
Length of train data: 10006
Length of test data: 1342

Data information:
Key parameters: ['movieMentions', 'respondentQuestions', 'messages', 'conversationId', 'respondentWorkerId', 'initiatorWorkerId', 'initiatorQuestions']
Key parameters in Questions: ['suggested', 'seen', 'liked']
Key parameters in messages: ['timeOffset', 'text', 'senderWorkerId', 'messageId']

Context information:
Total mentioned movie number (train): 52918
Total mentioned movie number in unique (train): 6223
Total message number (train): 182150
Total mentioned movie number (test): 7154
Total mentioned movie number in unique (test): 2007
Total message number (test): 23952
Average mentioned movie numbers per conversation (train): 5.288626823905656
Average message numbers per conversation (train): 18.20407755346792
Average mentioned movie numbers per conversation (test): 5.330849478390462
Average message numbers per conversation (test): 17.847988077496275

10006
[[], [], [], [], [], [], [], [], []

Bert 활용

In [12]:
# 코사인 유사도에 기반하여 단어 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화합니다.
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [13]:
# 중복을 최소화하고 결과의 다양성을 극대화
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [14]:
# 3개의 단어 묶음인 단어구 추출
print(line2)
x={}
line4=[[] for _ in range(10)]
p = re.compile("[0-9]+")
for i in range(10):
  k=''
  for j in line2[i]:
    j=p.sub(" ",j)
    k+=" "+j
  line4[i]=k
print(line4)
n_gram_range = (3, 3)
stop_words = "english"
for i in range(10):
  print()
  print(i)
  count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([line4[i]])
  candidates = count.get_feature_names_out()
  print('keyword 다섯개만 출력 :',candidates[:5])
  model = SentenceTransformer('distilbert-base-nli-mean-tokens')
  doc_embedding = model.encode([line4[i]])
  candidate_embeddings = model.encode(candidates)
  top_n = 5
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
  print(keywords)
  print("단어 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화")
  # nr_candidates값이 높을수록 더 다양한 키워드
  print(max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20))
  print("중복을 최소화하고 결과의 다양성을 극대화")
  #  diversity값이 높을수록 더 다양한 키워드
  print(mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7))

[['hi there how are you i m looking for movie recommendations', 'i am doing okay what kind of movies do you like ', 'i like animations like 84779 and 191602', 'i also enjoy 122159', 'anything artistic', 'you might like 165710 that was a good movie ', 'what s it about ', 'it has alec baldwin it is about a baby that works for a company and gets adopted it is very funny', 'that seems like a nice comedy', 'do you have any animated recommendations that are a bit more dramatic like 151313 for example', 'i like comedies but i prefer films with a little more depth', 'that is a tough one but i will remember something', ' 203371 was a good one', 'ooh that seems cool thanks for the input i m ready to submit if you are ', 'it is animated sci fi and has action', 'glad i could help', 'nice', 'take care cheers ', 'bye'], ['hi did you see 196336 ', 'yes it was a pretty good movie ', 'then you would like 114851 if you haven t seen it ', 'you like sci fi stuff like 204292 ', 'i am more of a 143189 kind 

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

['gets adopted funny', 'like good movie', 'nice comedy animated', 'comedies prefer films', 'good movie alec']
단어 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화
['glad help nice', 'prefer films little', 'movie recommendations doing', 'comedy animated recommendations', 'gets adopted funny']
중복을 최소화하고 결과의 다양성을 극대화
['good movie alec', 'baby works company', 'comedy animated recommendations', 'prefer films little', 'help nice care']

1
keyword 다섯개만 출력 : ['blood gore movies' 'cartoon movies like' 'claymation movie sounds'
 'click hide movie' 'clones claymation movie']
['movie perfect fan', 'wars movie perfect', 'pretty good movie', 'parody movies pretty', 'movies pretty funny']
단어 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화
['blood gore movies', 'did star wars', 'clones claymation movie', 'good tech film', 'perfect fan films']
중복을 최소화하고 결과의 다양성을 극대화
['movies pretty funny', 'wars clones claymation', 'star wars movie', 'like blood gore', 'fan films creep']

2
keyword 다섯개만 출력 : ['day thanks nice' 'excited upcoming favorit